In [ ]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-fbcytdvw
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-fbcytdvw
  Resolved https://github.com/keras-team/keras-preprocessing.git to commit 3e380065d4afc7347aaee8d89325a16b22158438
  Preparing metadata (setup.py) ... done
  Created wheel for Keras-Preprocessing: filename=Keras_Preprocessing-1.1.2-py3-none-any.whl size=43617 sha256=3c580b87a31e8ca869261b3845cb80c784179eef497c4266dd4503642fd2edfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-7hbubino/wheels/72/0b/c7/3f6b26f2d789c712867e02502e1f1b740091f8615f32a9f870
Successfully built Keras-Preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
training_set = '/content/drive/MyDrive/DATASET/training_set'
test_set = '/content/drive/MyDrive/DATASET/test_set'
prediction = '/content/drive/MyDrive/DATASET/prediction'

In [ ]:
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        training_set,
        target_size= (224, 224),
        batch_size=32,
        class_mode='binary')

Found 39123 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        test_set,
        target_size=(224,224),
        batch_size=32,
        class_mode='binary')

Found 17255 images belonging to 2 classes.


In [ ]:
from keras.utils import to_categorical
training_set_labels = to_categorical(training_set.labels)
test_set_labels = to_categorical(test_set.labels)

In [ ]:
num_classes = 2

In [ ]:
img_size = (224, 224)

In [ ]:
base_model = tf.keras.applications.MobileNet(input_shape=(224, 224, 3),
                                             include_top=False,
                                             weights='imagenet')
base_model.trainable = False

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(training_set,
                    epochs=10,
                    validation_data=test_set)


Epoch 1/10
1223/1223 [==============================] - 12486s 10s/step - loss: 0.2517 - accuracy: 0.9621 - val_loss: 0.1962 - val_accuracy: 0.9618
Epoch 2/10
1223/1223 [==============================] - 670s 548ms/step - loss: 0.1082 - accuracy: 0.9896 - val_loss: 0.1083 - val_accuracy: 0.9790
Epoch 3/10
1223/1223 [==============================] - 670s 548ms/step - loss: 0.0614 - accuracy: 0.9928 - val_loss: 0.0824 - val_accuracy: 0.9816
Epoch 4/10
1223/1223 [==============================] - 671s 549ms/step - loss: 0.0390 - accuracy: 0.9949 - val_loss: 0.0851 - val_accuracy: 0.9770
Epoch 5/10
1223/1223 [==============================] - 670s 548ms/step - loss: 0.0279 - accuracy: 0.9956 - val_loss: 0.0814 - val_accuracy: 0.9752
Epoch 6/10
1223/1223 [==============================] - 671s 549ms/step - loss: 0.0198 - accuracy: 0.9967 - val_loss: 0.0862 - val_accuracy: 0.9707
Epoch 7/10
1223/1223 [==============================] - 670s 548ms/step - loss: 0.0157 - accuracy: 0.9969 - val_

In [ ]:
model.save('/content/drive/MyDrive/DATASET/violence_detection_model.h5')

In [ ]:
#WITHOUT KEY FRAME EXTRACTION

import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model

# Load saved model
model = load_model('/content/drive/MyDrive/DATASET/violence_detection_model.h5')

# Define function to classify frame as violent or non-violent using model
def classify_frame(frame, model):
    img = cv2.resize(frame, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    if pred[0][0] > 0.5:
        return 'violence'
    else:
        return 'non-violence'

# Define input video path and output directories for violent and non-violent frames
input_video_path = '/content/drive/MyDrive/DATASET/prediction/input/hospital.mp4'
violence_dir = '/content/drive/MyDrive/DATASET/prediction/violence/frames0.1'
non_violence_dir = '/content/drive/MyDrive/DATASET/prediction/non-violence/frames0.1'

# Open video file
cap = cv2.VideoCapture(input_video_path)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Classify frame and save in appropriate directory
    classification = classify_frame(frame, model)
    frame_name = f"frame_{frame_count:05d}.jpg"
    if classification == 'violence':
        cv2.imwrite(os.path.join(violence_dir, frame_name), frame)
    else:
        cv2.imwrite(os.path.join(non_violence_dir, frame_name), frame)

# Release video file and print statistics
cap.release()
cv2.destroyAllWindows()
print(f"Total frames processed: {frame_count}")
print(f"Violent frames saved: {len(os.listdir(violence_dir))}")
print(f"Non-violent frames saved: {len(os.listdir(non_violence_dir))}")


In [ ]:
#WITH KEY FRAME EXTRACTION

import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model

# Load saved model
model = load_model('/content/drive/MyDrive/DATASET/violence_detection_model.h5')

# Define function to classify frame as violent or non-violent using model
def classify_frame(frame, model):
    img = cv2.resize(frame, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    if pred[0][0] > 0.5:
        return 'violence'
    else:
        return 'non-violence'

# Define input video path and key frame interval
input_video_path = '/content/drive/MyDrive/DATASET/prediction/input/V_313.mp4'
key_frame_interval = 10

# Define output directories for violent and non-violent frames
violence_dir = '/content/drive/MyDrive/DATASET/prediction/violence/frame000'
non_violence_dir = '/content/drive/MyDrive/DATASET/prediction/non-violence/frame000'

# Open video file
cap = cv2.VideoCapture(input_video_path)
frame_count = 0
prev_key_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count == 1:
        prev_key_frame = frame
        key_frame = frame
    elif frame_count % key_frame_interval == 0:
        mae = np.mean(cv2.absdiff(frame, prev_key_frame))
        if mae > 0.05:
            prev_key_frame = frame
            key_frame = frame
        else:
            continue
    else:
        continue

    # Classify key frame and save in appropriate directory
    classification = classify_frame(key_frame, model)
    frame_name = f"key_frame_{frame_count:05d}.jpg"
    if classification == 'violence':
        cv2.imwrite(os.path.join(violence_dir, frame_name), key_frame)
    else:
        cv2.imwrite(os.path.join(non_violence_dir, frame_name), key_frame)

# Release video file and print statistics
cap.release()
cv2.destroyAllWindows()
print(f"Total frames processed: {frame_count}")
print(f"Violent frames saved: {len(os.listdir(violence_dir))}")
print(f"Non-violent frames saved: {len(os.listdir(non_violence_dir))}")


1/1 [==============================] - 0s 21ms/step
Total frames processed: 156
Violent frames saved: 16
Non-violent frames saved: 0
